In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from scipy.stats import skew
from collections import OrderedDict
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import RFE
train_dataset=pd.read_csv('/home/navodini/Documents/NUS/Brats19/Brats_valid/radiomics_normalized.csv', header=0)
test_dataset=pd.read_csv('/home/navodini/Documents/NUS/Brats19/Brats_valid/radiomics_valid_normalized.csv', header=0)
every_column_except_y= [col for col in train_dataset.columns if col not in ['Patient_ID','class','Survival','Survival_indays']]
every_column_except_ID_test= [col for col in test_dataset.columns if col not in ['Patient_ID']]

model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42) 
rfe=RFE(model,n_features_to_select=14,step=1)
rfe.fit(train_dataset[every_column_except_y],train_dataset['Survival'])
ranking_RFE=rfe.ranking_
indices=np.where(ranking_RFE==1)
indices = list(indices[0])
data_RFE=train_dataset[every_column_except_y].iloc[:,indices]
data_RFE_test=test_dataset[every_column_except_ID_test].iloc[:,indices]

train_x=np.squeeze(data_RFE) 
test_x=np.squeeze(data_RFE_test)
train_y=train_dataset['Survival']

model.fit(train_x,train_y)

OrderedDict(sorted(model.get_booster().get_fscore().items(), key=lambda t: t[1], reverse=True))

most_relevant_features= list( dict((k, v) for k, v in model.get_booster().get_fscore().items() if v >= 50).keys())

train_X=train_x[most_relevant_features]

parameters_for_testing = {
   'colsample_bytree':[0.4,0.6,0.8],
   'gamma':[0,0.03,0.1,0.3],
   'min_child_weight':[1.5,6,10],
   'learning_rate':[0.1,0.07],
   'max_depth':[3,5],
   'n_estimators':[10000],
   'reg_alpha':[1e-5, 1e-2,  0.75],
   'reg_lambda':[1e-5, 1e-2, 0.45],
   'subsample':[0.6,0.95]  
}

                    
xgb_model = xgboost.XGBRegressor(learning_rate =0.1, n_estimators=1000, max_depth=5,
    min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=6, scale_pos_weight=1, seed=27)

gsearch1 = GridSearchCV(estimator = xgb_model, param_grid = parameters_for_testing, n_jobs=6,iid=False, verbose=10,scoring='neg_mean_squared_error')
gsearch1.fit(train_X,train_y)

print (gsearch1.cv_results_)
print('best params')
print (gsearch1.best_params_)
print('best score')
print (gsearch1.best_score_)

/home/navodini/py3_kernel/lib/python3.5/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/navodini/py3_kernel/lib/python3.5/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Fitting 3 folds for each of 2592 candidates, totalling 7776 fits


/home/navodini/py3_kernel/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   20.4s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   21.0s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   45.1s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   56.1s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  1.5min
[Parallel(n_jobs=6)]: Done  49 tasks      | elapsed:  1.8min
[Parallel(n_jobs=6)]: Done  60 tasks      | elapsed:  2.0min
[Parallel(n_jobs=6)]: Done  73 tasks      | elapsed:  2.6min
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  2.9min
[Parallel(n_jobs=6)]: Done 

{'param_gamma': masked_array(data=[0, 0, 0, ..., 0.3, 0.3, 0.3],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'std_score_time': array([0.04579907, 0.00036525, 0.00229365, ..., 0.00093712, 0.01464201,
       0.00300048]), 'split2_train_score': array([-1.31129328e-07, -3.33445755e-07, -1.31525883e-07, ...,
       -4.13377143e-02, -3.96123156e-02, -4.34881722e-02]), 'param_subsample': masked_array(data=[0.6, 0.95, 0.6, ..., 0.95, 0.6, 0.95],
             mask=[False, False, False, ..., False, False, False],
       fill_value='?',
            dtype=object), 'split1_train_score': array([-1.22414432e-07, -3.10313861e-07, -1.26477552e-07, ...,
       -4.10263687e-02, -3.96693479e-02, -4.14958716e-02]), 'mean_fit_time': array([11.29374949, 11.29347603, 12.05168017, ..., 34.1875751 ,
       28.81832091, 18.86581182]), 'split1_test_score': array([-0.13262213, -0.15219945, -0.12872748, ..., -0.13599722,
       -0.12694923, -0

In [3]:
best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.8,
                 gamma=0.3,                 
                 learning_rate=0.1,
                 max_depth=5,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.01,
                 reg_lambda=0.45,
                 subsample=0.95,
                 seed=42) 
best_xgb_model.fit(train_X,train_y)
test_dataset['Prediction'] = best_xgb_model.predict(test_dataset[most_relevant_features])
filename = 'submission_best_14.csv'
pd.DataFrame({'Patient_ID': test_dataset.Patient_ID, 'Survival': test_dataset.Prediction*1000}).to_csv(filename, index=False)

In [2]:
most_relevant_features

['EigenValues2_wt',
 'EigenValues1_tc',
 'FirstAxis3_tc',
 'FirstAxisLength_nec',
 'Centroid2_nec',
 'SecondAxis2_wt',
 'histogram_enhancement',
 'MeridionalEccentricity_wt',
 'SecondAxis2_tc',
 'Age',
 'entropy_necrosis',
 'histogram_necrosis',
 'EigenValues3_nec',
 'EigenValues2_tc']

In [4]:
test_dataset_new=pd.read_csv('/home/navodini/Documents/NUS/Brats19/Brats_valid/radiomics_test_normalized.csv', header=0)
every_column_except_ID_test_new= [col for col in test_dataset.columns if col not in ['Patient_ID']]
test_dataset_new['Prediction'] = best_xgb_model.predict(test_dataset_new[most_relevant_features])
filename = 'submission_best_14_test.csv'
pd.DataFrame({'Patient_ID': test_dataset_new.Patient_ID, 'Survival': test_dataset_new.Prediction*1000}).to_csv(filename, index=False)